## Setup

Load libraries

In [1]:
import pandas as pd
from snowai.density import (
    JonasDensity, 
    PistochiDensity, 
    SturmDensity,
    MachineLearningDensity
)


## Let's create sample data